Import Statements

In [1]:
from quantrocket.license import set_license
from quantrocket.history import create_usstock_db, collect_history
from quantrocket.master import create_universe, get_securities
from quantrocket import get_prices
import pandas as pd

License Key

In [2]:
set_license("eb9e3a59-de27-11ee-b787-f1b66bdc06a3")

{'licensekey': 'eb9e........06a3', 'software_license': {}}

Universe Creation

In [3]:
create_universe("apple", sids=["FIBBG000B9XRY4"])

{'code': 'apple', 'provided': 1, 'inserted': 1, 'total_after_insert': 1}

History Database Creation

In [4]:
create_usstock_db("apple-1d", bar_size="1 day", free=True)

{'status': 'successfully created quantrocket.v2.history.apple-1d.sqlite'}

History Data Collection

In [5]:
collect_history("aapl")

{'status': 'the historical data will be collected asynchronously'}

Securities Collection

In [6]:
securities = get_securities(vendors="usstock", sec_types="STK")

In [7]:
securities.head()

,Symbol,Exchange,Country,Currency,SecType,Etf,Timezone,Name,PriceMagnifier,Multiplier,Delisted,DateDelisted,LastTradeDate,RolloverDate
Sid,,,,,,,,,,,,,,
FIBBG000B9XRY4,AAPL,XNAS,US,USD,STK,False,America/New_York,APPLE INC,1,1,False,NaT,NaT,NaT
FIBBG000BFWKC0,MON,XNYS,US,USD,STK,False,America/New_York,MONSANTO CO,1,1,True,2018-06-06,NaT,NaT
FIBBG000BKZB36,HD,XNYS,US,USD,STK,False,America/New_York,HOME DEPOT INC,1,1,False,NaT,NaT,NaT
FIBBG000BMHYD1,JNJ,XNYS,US,USD,STK,False,America/New_York,JOHNSON & JOHNSON,1,1,False,NaT,NaT,NaT
FIBBG000BPH459,MSFT,XNAS,US,USD,STK,False,America/New_York,MICROSOFT CORP,1,1,False,NaT,NaT,NaT


Prices Data

Daily close price data of APPLE Stock for the year 2023

In [8]:
prices = get_prices("aapl", sids=["FIBBG000B9XRY4"], start_date="2023-01-01", end_date="2023-12-31", fields=["Close"])

In [9]:
prices.head()

Sid               FIBBG000B9XRY4
Field Date                      
Close 2023-01-03        124.2163
      2023-01-04        125.4975
      2023-01-05        124.1666
      2023-01-06        128.7352
      2023-01-09        129.2616

Pandas Dataframe Creation

In [10]:
prices = prices['FIBBG000B9XRY4']['Close'].to_list()
closes_df = pd.DataFrame({'Close': prices})

Returns Calculation

In [12]:
closes_df['Returns'] = closes_df.pct_change()

States Classification

In [14]:
def classify_state(r):
    if r >= 0.01:
        return 1  # Bull
    elif r > -0.01:
        return 0  # Flat
    else:
        return -1  # Bear

closes_df['State'] = closes_df['Returns'].apply(classify_state)

In [15]:
closes_df

,Close,Returns,State
0,124.2163,NaN,-1
1,125.4975,0.010314,1
2,124.1666,-0.010605,-1
3,128.7352,0.036794,1
4,129.2616,0.004089,0
...,...,...,...
245,193.3533,-0.005547,0
246,192.8040,-0.002841,0
247,192.9038,0.000518,0
248,193.3333,0.002226,0


Transition Matrix

In [16]:
transition_counts = {'Bull': {'Bull': 0, 'Flat': 0, 'Bear': 0},
                     'Flat': {'Bull': 0, 'Flat': 0, 'Bear': 0},
                     'Bear': {'Bull': 0, 'Flat': 0, 'Bear': 0}}
state_mapping = {1: 'Bull', 0: 'Flat', -1: 'Bear'}

for i in range(1, len(closes_df)):
    current_state = closes_df.iloc[i]['State']
    previous_state = closes_df.iloc[i-1]['State']
    transition_counts[state_mapping[previous_state]][state_mapping[current_state]] += 1
    
transition_counts

{'Bull': {'Bull': 11, 'Flat': 38, 'Bear': 7},
 'Flat': {'Bull': 40, 'Flat': 94, 'Bear': 23},
 'Bear': {'Bull': 5, 'Flat': 26, 'Bear': 5}}

Transition Probabilities

In [17]:
transition_probabilities = {}
for origin_state, transitions in transition_counts.items():
    total_transitions = sum(transitions.values())
    # Normalize the transition counts by the total transitions from each state to calculate probabilities
    transition_probabilities[origin_state] = {target_state: count / total_transitions for target_state, count in transitions.items() if total_transitions > 0}

transition_probabilities

{'Bull': {'Bull': 0.19642857142857142,
  'Flat': 0.6785714285714286,
  'Bear': 0.125},
 'Flat': {'Bull': 0.25477707006369427,
  'Flat': 0.5987261146496815,
  'Bear': 0.1464968152866242},
 'Bear': {'Bull': 0.1388888888888889,
  'Flat': 0.7222222222222222,
  'Bear': 0.1388888888888889}}

Portfolio Value and Buy Indices

In [19]:
portfolio_value = 0
buy_indices = []

for i in range(1, len(closes_df) - 1):
    current_state = closes_df.iloc[i]['State']
    next_state = closes_df.iloc[i+1]['State']
    if current_state == 0 and next_state == 1:  # Flat to Bull transition
        portfolio_value += 1
        buy_indices.append(i+1)
    elif current_state == 0 and next_state == -1:  # Flat to Bear transition
        portfolio_value -= 1

print("Portfolio Value = ", portfolio_value)
print("Buy Indices: ", buy_indices)

Portfolio Value =  17
Buy Indices:  [6, 8, 12, 16, 21, 28, 30, 41, 50, 52, 59, 61, 69, 79, 85, 88, 94, 100, 103, 108, 110, 113, 117, 120, 123, 133, 142, 160, 164, 177, 187, 191, 207, 209, 212, 216, 218, 232, 234, 238]
